Bật tính năng lưu tạm thời cho những phần đã gen khi gọi hàm generate của @learningQNA.js @learningMeaning.js @learningCard.js @learningFlexible.js 
------------
tôi nghĩ là nên làm 1 file .js riêng cùng cấp với các file này

|               | Cache (Memory)                        | SessionStorage                           |
|---------------|--------------------------------------|------------------------------------------|
| **Ưu điểm**   | - Tốc độ truy xuất nhanh nhất        | - Dữ liệu được giữ khi refresh page      |
|               | - Đơn giản, dễ implement             | - Dung lượng lớn hơn (5-10MB)            |
|               | - Không phụ thuộc vào browser storage| - Tách biệt dữ liệu giữa các tab         |
|               |                                      | - Tự động clear khi đóng tab/window      |
| **Nhược điểm**| - Mất dữ liệu khi refresh page       | - Chậm hơn memory cache một chút         |
|               | - Không lưu được giữa các tab        | - Phụ thuộc vào browser storage          |
|               | - Chiếm RAM của client               |                                          |
| **Phù hợp**   | Khi yêu cầu tốc độ truy xuất nhanh   | - Khi cần giữ dữ liệu khi refresh page   |
|               | và không cần lưu dữ liệu lâu dài     | - Dữ liệu không quá lớn                  |
|               |                                      | - Cần tách biệt dữ liệu giữa các phiên   |


Viết cache.js cho @learningQNA.js @learningCard.js @learningMeaning.js @learningFlexible.js 
- Viết 1 file bộ nhớ tạm thời cho các phần này . 

- Sau khi ấn Generatequestion ở file @generateQuestion.js 
thì xóa data tạm thời của 4 file trên. 
Tuy nhiên sau khi @learningMeaning.js được kích hoạt => sẽ lưu tạm thời vào phiên đó, 
Sau đó user chuyển qua @learningCard.js , @learningFlexible.js , ... => Khi quay lại @generateQuestion.js  thì vẫn còn nguyên (tức ko cần chạy lại việc gen dữ liệu = cách call về @generateQuestionsController.js )

I'll help you create a cache.js file to manage temporary storage for the learning modules. Here's the implementation:

```javascript:frontend/src/js/modules/cache.js
class LearningCache {
    constructor() {
        this.cache = {
            qna: null,
            card: null,
            meaning: null,
            flexible: null
        };
    }

    // Clear all cached data
    clearAll() {
        this.cache = {
            qna: null,
            card: null,
            meaning: null,
            flexible: null
        };
    }

    // Set cache for specific module
    set(module, data) {
        if (this.cache.hasOwnProperty(module)) {
            this.cache[module] = data;
        }
    }

    // Get cache for specific module
    get(module) {
        return this.cache[module];
    }

    // Check if module has cached data
    has(module) {
        return this.cache[module] !== null;
    }
}

// Create singleton instance
const learningCache = new LearningCache();
export default learningCache;
```

Now, let's modify the relevant parts of the other files to use this cache:

```javascript:frontend/src/js/generateQuestion.js
// ... existing imports ...
import learningCache from './modules/cache.js';

async function handleGenerateClick() {
    // Clear all cached data when generating new questions
    learningCache.clearAll();
    
    // ... rest of existing code ...
}
```

```javascript:frontend/src/js/modules/learningMeaning.js
// ... existing imports ...
import learningCache from './cache.js';

async function generateLearningMeaning(lessons) {
    try {
        // Check cache first
        if (learningCache.has('meaning')) {
            const cachedData = learningCache.get('meaning');
            displayLearningMeaningResults(cachedData);
            return;
        }

        currentLessonId = storagedLessons?.[0]?.lesson_id || generateUniqueId();

        showLoadingDialog();
        const response = await fetch(`${API_URL}/generate-learning-meaning`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ lessons })
        });
        
        const data = await response.json();
        rawApiResponse = data;
        learningMeaningLessons = data.map(item => ({
            ...item,
            lesson_id: currentLessonId
        }));

        // Cache the results
        learningCache.set('meaning', learningMeaningLessons);
        
        displayLearningMeaningResults(learningMeaningLessons);
    } catch (error) {
        console.error('Error:', error);
        alert(error.message);
    } finally {
        hideLoadingDialog();
    }
}
```

Similar modifications for the other learning modules:

```javascript:frontend/src/js/modules/learningCard.js
// ... existing imports ...
import learningCache from './cache.js';

async function generateLearningCard(lessons) {
    try {
        // Check cache first
        if (learningCache.has('card')) {
            const cachedData = learningCache.get('card');
            displayLearningCardResults(cachedData);
            return;
        }

        // ... existing code ...

        // Cache the results
        learningCache.set('card', learningCardLessons);
        
        displayLearningCardResults(learningCardLessons);
    } catch (error) {
        // ... error handling ...
    }
}
```

```javascript:frontend/src/js/modules/learningQNA.js
// ... existing imports ...
import learningCache from './cache.js';

async function generateLearningQNA(lessons) {
    try {
        // Check cache first
        if (learningCache.has('qna')) {
            const cachedData = learningCache.get('qna');
            displayLearningQNAResults(cachedData);
            return;
        }

        // ... existing code ...

        // Cache the results
        learningCache.set('qna', learningQNALessons);
        
        displayLearningQNAResults(data);
    } catch (error) {
        // ... error handling ...
    }
}
```

```javascript:frontend/src/js/modules/learningFlexible.js
// ... existing imports ...
import learningCache from './cache.js';

async function generateLearningFlexible(lessons) {
    try {
        // Check cache first
        if (learningCache.has('flexible')) {
            const cachedData = learningCache.get('flexible');
            displayLearningFlexibleResults(cachedData);
            return;
        }

        // ... existing code ...

        // Cache the results
        learningCache.set('flexible', learningFlexibleLessons);
        
        displayLearningFlexibleResults(data);
    } catch (error) {
        // ... error handling ...
    }
}
```

This implementation:

1. Creates a centralized cache system using a singleton pattern
2. Clears all cached data when generating new questions
3. Checks for cached data before making API calls in each learning module
4. Stores results in cache after successful API calls
5. Maintains cache persistence until new questions are generated

The cache will persist across module switches until new questions are generated via handleGenerateClick(). This prevents unnecessary API calls while allowing users to switch between modules freely.

Tôi sẽ phân tích ưu nhược điểm của cả cache và session để bạn có thể chọn giải pháp phù hợp:

### Cache (Memory)
**Ưu điểm:**
- Tốc độ truy xuất nhanh nhất vì dữ liệu nằm trong memory
- Đơn giản, dễ implement
- Không phụ thuộc vào browser storage

**Nhược điểm:**
- Mất dữ liệu khi refresh page
- Không lưu được giữa các tab
- Chiếm RAM của client

### SessionStorage
**Ưu điểm:**
- Dữ liệu được giữ khi refresh page
- Dung lượng lớn hơn (5-10MB)
- Tách biệt dữ liệu giữa các tab
- Tự động clear khi đóng tab/window

**Nhược điểm:**
- Chậm hơn memory cache một chút
- Phụ thuộc vào browser storage

Với requirements của bạn, tôi nghĩ **SessionStorage** sẽ phù hợp hơn vì:

1. Cần giữ dữ liệu khi refresh page
2. Dữ liệu không quá lớn
3. Cần tách biệt dữ liệu giữa các phiên làm việc

Đây là cách implement với SessionStorage:

```javascript:frontend/src/js/modules/storage.js
class LearningStorage {
    constructor() {
        this.STORAGE_KEYS = {
            QNA: 'learning_qna',
            CARD: 'learning_card',
            MEANING: 'learning_meaning', 
            FLEXIBLE: 'learning_flexible'
        };
    }

    // Clear all stored data
    clearAll() {
        Object.values(this.STORAGE_KEYS).forEach(key => {
            sessionStorage.removeItem(key);
        });
    }

    // Set data for specific module
    set(module, data) {
        const key = this.STORAGE_KEYS[module.toUpperCase()];
        if (key) {
            sessionStorage.setItem(key, JSON.stringify(data));
        }
    }

    // Get data for specific module
    get(module) {
        const key = this.STORAGE_KEYS[module.toUpperCase()];
        const data = sessionStorage.getItem(key);
        return data ? JSON.parse(data) : null;
    }

    // Check if module has stored data
    has(module) {
        const key = this.STORAGE_KEYS[module.toUpperCase()];
        return sessionStorage.getItem(key) !== null;
    }
}

// Create singleton instance
const learningStorage = new LearningStorage();
export default learningStorage;
```

Cách sử dụng trong các module:

```javascript:frontend/src/js/modules/learningMeaning.js
// ... existing imports ...
import learningStorage from './storage.js';

async function generateLearningMeaning(lessons) {
    try {
        // Check storage first
        if (learningStorage.has('meaning')) {
            const storedData = learningStorage.get('meaning');
            displayLearningMeaningResults(storedData);
            return;
        }

        // ... existing API call code ...

        // Store the results
        learningStorage.set('meaning', learningMeaningLessons);
        
        displayLearningMeaningResults(learningMeaningLessons);
    } catch (error) {
        console.error('Error:', error);
        alert(error.message);
    }
}
```

```javascript:frontend/src/js/generateQuestion.js
// ... existing imports ...
import learningStorage from './modules/storage.js';

async function handleGenerateClick() {
    // Clear all stored data when generating new questions
    learningStorage.clearAll();
    
    // ... rest of existing code ...
}
```

Lợi ích khi dùng SessionStorage:
1. Dữ liệu được giữ lại khi user refresh page
2. Mỗi tab làm việc độc lập
3. Tự động clear khi đóng tab/window
4. Không tốn RAM của client
5. Dung lượng đủ lớn cho nhu cầu lưu trữ tạm thời

Bạn có thể test bằng cách:
1. Generate questions
2. Chuyển qua lại giữa các module
3. Refresh page
4. Mở tab mới
5. Đóng tab/window

SessionStorage sẽ hoạt động đúng trong tất cả các trường hợp trên.